# Brest Cancer Detection with PyTorch NN

## Imports

In [179]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load Data

In [180]:
X, y = load_breast_cancer(return_X_y=True)

### Split and Scale Data

In [181]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## 

In [182]:
X_train

array([[ 3.0287957 ,  0.58190223,  3.08097762, ...,  2.04090688,
         0.36131005,  0.69974369],
       [-1.21647338,  2.1334791 , -1.22259114, ..., -0.97191616,
        -0.91392327, -0.64044058],
       [-0.94391997,  0.29744647, -0.89843484, ..., -0.4061406 ,
        -0.48071004,  0.17129705],
       ...,
       [-0.37598145, -1.63497199, -0.42670865, ..., -0.46311211,
        -0.22139226,  0.58697209],
       [ 0.16627141,  0.97919994,  0.11590082, ...,  0.34994404,
        -0.67596107, -0.84473696],
       [-0.38454334, -0.42897361, -0.41137133, ..., -0.86039746,
        -0.35410194, -0.81640786]], shape=(386, 30))

In [183]:
y_train

array([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,

### Convert to Tensors

In [184]:
X_train_tensor = torch.from_numpy(X_train).float()
X_val_tensor = torch.from_numpy(X_val).float()
X_test_tensor = torch.from_numpy(X_test).float()

y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)
y_val_tensor = torch.from_numpy(y_val).float().unsqueeze(1)
y_test_tensor = torch.from_numpy(y_test).float().unsqueeze(1)

In [185]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

## Model

In [186]:
class BCNet(nn.Module):
    def __init__(self):
        super(BCNet, self).__init__()
        self.fc1 = nn.Linear(30, 16)
        self.bn1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.bn2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.bn1(self.fc1(x))))
        x = self.dropout(F.relu(self.bn2(self.fc2(x))))
        x = self.fc3(x)
        return x

In [187]:
model = BCNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

### Training

In [188]:
epochs = 100  # set high — early stopping will handle when to stop
patience = 10
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(epochs):
    # Training
    model.train()
    running_loss = 0.0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = model(x_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_logits = model(X_val_tensor)
        val_loss = criterion(val_logits, y_val_tensor).item()
    
    scheduler.step(val_loss)
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# Restore best model
model.load_state_dict(best_model_state)

Epoch 1/100, Train Loss: 0.7360, Val Loss: 0.6684, LR: 0.001000
Epoch 2/100, Train Loss: 0.6928, Val Loss: 0.5975, LR: 0.001000
Epoch 3/100, Train Loss: 0.6232, Val Loss: 0.5637, LR: 0.001000
Epoch 4/100, Train Loss: 0.5890, Val Loss: 0.5192, LR: 0.001000
Epoch 5/100, Train Loss: 0.5781, Val Loss: 0.4925, LR: 0.001000
Epoch 6/100, Train Loss: 0.5556, Val Loss: 0.4724, LR: 0.001000
Epoch 7/100, Train Loss: 0.4846, Val Loss: 0.4658, LR: 0.001000
Epoch 8/100, Train Loss: 0.5318, Val Loss: 0.4441, LR: 0.001000
Epoch 9/100, Train Loss: 0.4937, Val Loss: 0.4263, LR: 0.001000
Epoch 10/100, Train Loss: 0.4796, Val Loss: 0.4157, LR: 0.001000
Epoch 11/100, Train Loss: 0.4555, Val Loss: 0.3982, LR: 0.001000
Epoch 12/100, Train Loss: 0.4186, Val Loss: 0.3948, LR: 0.001000
Epoch 13/100, Train Loss: 0.3895, Val Loss: 0.4099, LR: 0.001000
Epoch 14/100, Train Loss: 0.4019, Val Loss: 0.4058, LR: 0.001000
Epoch 15/100, Train Loss: 0.3783, Val Loss: 0.3740, LR: 0.001000
Epoch 16/100, Train Loss: 0.3461, 

<All keys matched successfully>

### Evaluation

In [189]:
with torch.no_grad():
    model.eval()
    logits = model(X_test_tensor)
    loss = criterion(logits, y_test_tensor).item()
    predictions = torch.sigmoid(logits)
    accuracy = ((predictions >= 0.5) == y_test_tensor).float().mean().item()
    
    print("Accuracy: ", accuracy)
    print("Loss: ", loss)

Accuracy:  0.9824561476707458
Loss:  0.11932429671287537
